**Important:** The model estimation code is intended to work with an experimental parallelised Vensim engine. With appropriate modifications to the main function calls (but not the analytical procedure), the same analysis can be run on regular commercially available Vensim DSS, though it will take *much* longer. Please contact [Tom Fiddaman](mailto:tom@ventanasystems.com) for information on the experimental Vensim engine.

For more information on the model estimation procedure, see S4 of the Supplementary Materials of the paper.

**Note:** if running in Jupyter, the `keyboard` module may need to be installed directly in the Notebook; see [here](https://stackoverflow.com/questions/38368318/installing-a-pip-package-from-within-a-jupyter-notebook-not-working) for example. The keyboard module is *only* used to bypass Vengine error messages; if not running Vengine (e.g. using normal Vensim DSS), it is not necessary, and you can safely remove the import statement and all press commands in the code.

---

This notebook contains code cells for the following:
1. Class & function definitions
2. Controlfile input & parsing
3. Main iterative estimation process & MCMC
4. Baseline projections & sensitivity analysis of input projection assumptions
5. Policy scenarios w/ uncertainty quantification
6. Loop knockout (structural sensitivity) analyses
7. Parametric assumption elasticity analyses
8. Alternative data condition analyses
9. Holdout data (out-of-sample validation) analysis
10. Synthetic data generation & analysis
11. PSRF statistic parsing

The first three of these (definitions, controlfile input, main estimation) should always be run first and in order (using controlfile bypass settings to skip main estimation if not needed). Remaining analyses can be run as needed, and do not need to be in order.

In [ ]:
import os
import subprocess
import regex
import json
import time
import numpy as np
import pandas as pd
from keyboard import press
from shutil import copy
from distutils.dir_util import copy_tree
from statsmodels.tsa.tsatools import detrend

In [ ]:
class Script(object):
    """Master object for holding and modifying .cmd script settings, 
    creating .cmd files, and running them through Vensim/Vengine"""
    def __init__(self, controlfile):
        print("Initialising", self)
        for k, v in controlfile['simsettings'].items():
            self.__setattr__(k, v if isinstance(v, str) else v.copy())
        self.setvals = []
        self.runcmd = "SIMULATE>REPORT|1\nMENU>RUN_OPTIMIZE|o\n"
        self.savecmd = f"MENU>VDF2TAB|!|!|{self.savelist}|\n"
        self.basename = controlfile['baserunname']
        self.cmdtext = []
        
    def copy_model_files(self, dirname):
        """Create subdirectory and copy relevant model files to it,
        then change working directory to subdirectory"""
        os.makedirs(dirname, exist_ok=True)
        os.chdir(f"./{dirname}")

        # Copy needed files from the working directory into the sub-directory
        for s in ['model', 'payoff', 'optparm', 'sensitivity', 'savelist', 'senssavelist']:
            if getattr(self, s):
                copy(f"../{getattr(self, s)}", "./")
        for slist in ['data', 'changes']:
            for file in getattr(self, slist):
                copy(f"../{file}", "./")
        
    def add_suffixes(self, settingsfxs):
        """Modify mdl, voc, etc. with suffixes specified as dict"""
        for s, sfx in settingsfxs.items():
            if hasattr(self, s):
                self.__setattr__(s, getattr(self, s)[:-4] + sfx + getattr(self, s)[-4:])
   
    def update_changes(self, chglist, setvals=[]):
        """Combines and flattens list of paired names & suffixes for 
        changes, and appends to changes setting; also updates setvals"""
        flat = [i for s in 
                [[f"{self.basename}_{n}_{sfx}.out" for n in name] if isinstance(name, list) 
                 else [f"{self.basename}_{name}_{sfx}.out"] for name, sfx in chglist] for i in s]
        self.changes.extend(flat)
        self.setvals = setvals
        
    def write_script(self, scriptname):
        """Write actual .cmd file based on controlfile attributes"""
        self.cmdtext.extend(["SPECIAL>NOINTERACTION\n", 
                             f"SPECIAL>LOADMODEL|{self.model}\n"])
        
        for s in ['payoff', 'sensitivity', 'optparm', 'savelist', 'senssavelist']:
            if hasattr(self, s):
                self.cmdtext.append(f"SIMULATE>{s}|{getattr(self, s)}\n")
        
        if hasattr(self, 'data'):
            datatext = ','.join(self.data)
            self.cmdtext.append(f"SIMULATE>DATA|\"{','.join(self.data)}\"\n")

        if hasattr(self, 'changes'):
            self.cmdtext.append(f"SIMULATE>READCIN|{self.changes[0]}\n")
            for file in self.changes[1:]:
                self.cmdtext.append(f"SIMULATE>ADDCIN|{file}\n")
        
        self.cmdtext.extend(["\n", f"SIMULATE>RUNNAME|{scriptname}\n"])
        
        if hasattr(self, 'setvals'):
            for var, val in self.setvals:
                self.cmdtext.append(f"SIMULATE>SETVAL|{var}={val}\n")
        
        self.cmdtext.extend([self.runcmd, self.savecmd, 
                             "SPECIAL>CLEARRUNS\n", "MENU>EXIT\n"])
        
        with open(f"{scriptname}.cmd", 'w') as scriptfile:
            scriptfile.writelines(self.cmdtext)

    def run_script(self, scriptname, controlfile, subdir, logfile):
        """Run the compiled .cmd file, calling Vengine by default"""
        return run_vengine_script(scriptname, controlfile['vensimpath'], 
                                  controlfile['timelimit'], '.log', check_opt, logfile)
        

class SubdirScript(Script):
    """Script subclass for optimization runs in subdirectory"""
    def prep_subdir(self, scriptname, controlfile, subdir):
        self.copy_model_files(subdir)
        copy(f"../{scriptname}.cmd", "./")

    def run_script(self, scriptname, controlfile, subdir, logfile):
        """Copy necessary files to subdirectory and run there, copying 
        .out file back to parent directory when complete"""
        self.prep_subdir(scriptname, controlfile, subdir)
        payoff = run_vengine_script(scriptname, controlfile['vensimpath'], 
                                    controlfile['timelimit'], '.log', check_opt, logfile)
        copy(f"./{scriptname}.out", "..") # Copy the .out file to parent directory
        os.chdir("..")
        return payoff


class MCScript(Script):
    """Script subclass for MCMC optimizations"""
    def run_script(self, scriptname, controlfile, subdir, logfile):
        run_vengine_script(scriptname, controlfile['vensimpath'], 
                           controlfile['timelimit'], '_MCMC_points.tab', check_MC, logfile)
        
        downsample(scriptname, controlfile['samplefrac'], remove=True) # Create MCMC subsample

        
class LongScript(Script):
    """Script subclass for long calibration runs e.g. all-params"""
    def run_script(self, scriptname, controlfile, subdir, logfile):
        return run_vengine_script(scriptname, controlfile['vensimpath'], 
                                  controlfile['timelimit']*5, '.log', check_opt, logfile)

    
class RunScript(Script):
    """Script subclass for simple single runs (not optimzations)"""
    def __init__(self, controlfile):
        super().__init__(controlfile)
        self.runcmd = "MENU>RUN|o\n" # Change .cmd run command

    def run_script(self, scriptname, controlfile, subdir, logfile):
        """Run the compiled .cmd file with Vensim instead of Vengine"""
        return run_vensim_script(scriptname, controlfile['vensim7path'], 10, logfile)
            

class ScenScript(Script):
    """Script subclass for scenario analysis with .cin files"""
    def update_changes(self, chglist, setvals=None):
        scen = []
        try: # Pull out all str items from end of chglist
            while type(chglist[-1]) == str:
                scen.append(chglist.pop())
        except IndexError: pass
        super().update_changes(chglist, setvals)
        scen.reverse() # Remember to reverse to maintain scen order!
        self.changes.extend(scen) # Then add str items to self.changes
        chglist.extend(scen) # And add them back to chglist as well
        
    def run_script(self, scriptname, controlfile, subdir, logfile):
        return run_vengine_script(scriptname, controlfile['vensimpath'], 
                                  controlfile['timelimit']/5, '.vdf', check_run, logfile)
    

class ScenRunScript(ScenScript):
    """Script subclass for scenario analysis runs (not optimizations)"""
    def __init__(self, controlfile):
        super().__init__(controlfile)
        self.runcmd = "MENU>RUN|o\n"


class ScenSensScript(ScenScript):
    """Script subclass for scenario sensitivity analysis"""
    def __init__(self, controlfile):
        super().__init__(controlfile)
        self.sensitivity = self.basename + '_full.vsc'
        self.runcmd = "MENU>RUN_SENSITIVITY|o\n"
        self.savecmd = f"MENU>SENS2FILE|!|!|%#T\n" # Exports only percentiles

        
class FullSensScript(ScenScript):
    """Script subclass for scenario sensitivity w/ full simulation export"""
    def __init__(self, controlfile):
        super().__init__(controlfile)
        self.sensitivity = self.basename + '_full.vsc'
        self.runcmd = "MENU>RUN_SENSITIVITY|o\n"
        self.savecmd = f"MENU>SENS2FILE|!|!|#T[\n" # Exports full sample (VERY LARGE!)
        
    def run_script(self, scriptname, controlfile, subdir, logfile):
        """Run with extended timelimit to allow time for vdf export"""
        return run_vengine_script(scriptname, controlfile['vensimpath'], 
                                  controlfile['timelimit']*5, '.log', check_run, logfile)


def compile_script(controlfile, scriptclass, name, namesfx, settingsfxs, 
                   logfile, chglist=[], setvals=[], subdir=None):
    """Master function for assembling & running .cmd script
    
    Parameters
    ----------
    controlfile : JSON object
        Master control file specifying sim settings, runname, etc.
    scriptclass : Script object
        Type of script object to instantiate, depending on run type
    name : str
    namesfx : str
        Along with `name`, specifies name added to baserunname for run
    settingsfxs : dict of str
        Dict of suffixes to append to filenames in simsettings; use to 
        distinguish versions of e.g. .mdl, .voc, .vpd etc. files
    logfile : str of filename/path
    chglist : list of tuples of (str or list, str)
        Specifies changes files to be used in script; specify as tuples 
        corresponding to `name`, `namesfx` of previous run .out to use; 
        tuples can also take a list of `names` as first element, taking 
        each with the same second element; if used with ScenScript run, 
        `chglist` can also take non-tuple strs as final elements, which 
        will be added directly (e.g. .cin files for scenarios)
    setvals : list of tuples of (str, int or float, <str>)
        Specifies variables and values to change for a given run using 
        Vensim's SETVAL script command; by default all SETVAL commands 
        will be implemented together for main run, but if `scriptclass` 
        is MultiScript, each SETVAL command will be implemented and run 
        separately in sequence; if used with MultiScript, each tuple in 
        `setvals` will require a third str element specifying the suffix 
        with which to save the run
    subdir : str, optional
        Name of subdirectory to create/use for run, if applicable
    
    Returns
    -------
    float
        Payoff value of the script run, if applicable, else 0
    """
    mainscript = scriptclass(controlfile)
    mainscript.add_suffixes(settingsfxs)
    mainscript.update_changes(chglist, setvals)
    scriptname = f"{mainscript.basename}_{name}_{namesfx}"    
    mainscript.write_script(scriptname)
    return mainscript.run_script(scriptname, controlfile, subdir, logfile)


def write_log(string, logfile):
    """Writes printed script output to a logfile"""
    with open(logfile,'a') as f:
        f.write(string + "\n")
    print(string)
    

def check_opt(scriptname, logfile):
    """Check function for use with run_vengine_script for optimizations"""
    if check_zeroes(scriptname):
        write_log(f"Help! {scriptname} is being repressed!", logfile)
    return not check_zeroes(scriptname)

def check_MC(scriptname, logfile, threshold=0.1):
    """Check function for use with run_vengine_script for MCMC"""
     # Check for discrepancy between .out and .rep payoff
    if abs(compare_payoff(scriptname)) >= threshold:
        write_log(f"{scriptname} is a self-perpetuating autocracy! re-running MC...", logfile)
        return False
    return True

def check_run(scriptname, logfile):
    """Check function for use with run_vengine_script for normal & sens runs"""
    if not os.path.exists(f"./{scriptname}.vdf"): # Check that output VDF exists
        write_log(f"Help! {scriptname} is being repressed!", logfile)
    return os.path.exists(f"./{scriptname}.vdf")


def run_vengine_script(scriptname, vensimpath, timelimit, checkfile, check_func, logfile):
    """Call Vengine with command script using subprocess; monitor output 
    file for changes to see if Vengine has stalled out, and restart if 
    it does, or otherwise bugs out; return payoff if applicable"""

    write_log(f"Initialising {scriptname}!", logfile)

    while True:
        proc = subprocess.Popen(f"{vensimpath} \"./{scriptname}.cmd\"")
        time.sleep(2)
        press('enter') # Necessary to bypass the popup message in Vengine
        while True:
            try: # See if run completes within timelimit
                proc.wait(timeout=timelimit)
                break
            except subprocess.TimeoutExpired: # If timelimit reached, check run status
                try: # If Vengine gives error popup on exit, attempt to bypass it
                    print("Attempting bypass...")
                    press('enter')
                    proc.wait(3) # Process should complete if bypass successful
                    break
                except subprocess.TimeoutExpired:
                    try: # If bypass unsuccessful, check if run still going
                        write_log(f"Checking for {scriptname}{checkfile}...", logfile)
                        timelag = time.time() - os.path.getmtime(f"./{scriptname}{checkfile}")                        
                        if timelag < (timelimit): # Compare time since last output with timelimit
                            write_log(f"At {time.ctime()}, {round(timelag,3)}s since last output, "
                                      "continuing...", logfile)
                            continue
                        else: # If run seems to have stalled out, kill and restart
                            proc.kill()
                            write_log(f"At {time.ctime()}, {round(timelag,3)}s since last output. "
                                      "Calibration timed out!", logfile)
                            break
                    except FileNotFoundError: # If check fails, kill and restart
                        proc.kill()
                        write_log("Calibration timed out!", logfile)
                        break
        # Check if process successfully completed or bugged out / was killed
        if proc.returncode != 1: # Note that Vengine returns 1 on MENU>EXIT, not 0!
            write_log(f"Return code is {proc.returncode}", logfile)
            write_log("Vensim! Trying again...", logfile)
            continue
        try: # If process completed successfully, run final check for errors in output
            if check_func(scriptname, logfile):
                break # NOTE: this is the only successful completion outcome!
        except FileNotFoundError: # Catch output error and restart run
            write_log("Outfile not found! That's it, I'm dead.", logfile)
            pass
    
    time.sleep(2)

    if os.path.exists(f"./{scriptname}.out"):
        payoffvalue = read_payoff(f"{scriptname}.out")
        write_log(f"Payoff for {scriptname} is {payoffvalue}, calibration complete!", logfile)
        return payoffvalue # For optimisation runs, return payoff
    return 0 # Set default payoff value for simtypes that don't generate one


def run_vensim_script(scriptname, vensim7path, maxattempts, logfile):
    """Call Vensim (not Vengine) with command script using subprocess; 
    for instances when Vengine not necessary; try up to `maxattempts` 
    times; return payoff if applicable"""
    attempts = 0
    while attempts < maxattempts:
        attempts += 1 # Track & update number of attempts to prevent infinite loop
        if os.path.exists(f"./{scriptname}.tab"):
            os.remove(f"./{scriptname}.tab") # Delete old output tabfile if needed
        try:
            subprocess.run(f"{vensim7path} \"./{scriptname}.cmd\"", check=True)
            pass
        except subprocess.CalledProcessError:
            print("Vensim! Trying again...")
            continue
        if os.path.exists(f"./{scriptname}.tab"): # Check for output tabfile
            break
        else:
            write_log(f"Help! {scriptname} is being repressed!", logfile)
            continue
    
    if os.path.exists(f"./{scriptname}.out"):
        payoffvalue = read_payoff(f"{scriptname}.out")
        write_log(f"Payoff for {scriptname} is {payoffvalue}, calibration complete!", logfile)
        return payoffvalue # For optimisation runs, return payoff
    return 0 # Set default payoff value for simtypes that don't generate one


def create_syn_mdl(modelname, newmodelname):
    """Open .mdl as text and modify as needed for syndata analysis"""
    with open(modelname,'r') as f:
        filedata = f.read()
        
    synvarregex = regex.compile(r"SynVar\[Elm\]=[\s\S]*?(\n\t~)")
    datavarregex = regex.compile(r"MaxDataTime,\s*DataVarBase\[Elm\]")
    
    # Switch SynVar to read data from file (from previously generated synthetic dataset)
    newdata = synvarregex.sub("SynVar[Elm]\n\t~", filedata)
    
    # Switch DataVar to read data from SynVar
    # DataVar default equation is `IF THEN ELSE(Time <= MaxDataTime, DataVarBase[Elm], :NA:)`
    newdata = datavarregex.sub("MaxDataTime, SynVar[Elm]", newdata)

    with open(newmodelname,'w') as f:
        f.write(newdata) # Save new .mdl for synthetic data calibration

                       
def split_voc(vocname, fractolfactor, linekey, mcsettings):
    """Splits .VOC file into multiple versions, for main, overdose, 
    initial values, and full model calibration"""
    with open(vocname,'r') as f0:
        filedata = f0.readlines()
    
    ### LINEKEY should include all OD-related parameter keywords e.g. fentanyl, overdose, OD
    
    vocmain = [line for line in filedata if line[0] == ':' or 
               ('Initial' not in line and not any(k in line for k in linekey))]
    vocod = [line for line in filedata if line[0] == ':' or any(k in line for k in linekey)]
    vocinit = [line for line in filedata if line[0] == ':' or 'Initial' in line]
    vocfull = filedata.copy()
    vocsens = vocfull.copy()
    vocmc = ''.join(vocfull)
    
    # Identify and multiply fractional tolerance by fractolfactor for initial & sens runs
    for l, line in enumerate(vocsens):
        if ':FRACTIONAL_TOLERANCE' in line:
            fractol = float(line.split('=')[1])
            vocsens[l] = f":FRACTIONAL_TOLERANCE={min(fractol*fractolfactor,0.1)}\n"

    # Make necessary substitutions for MCMC settings
    for k,v in mcsettings.items():
        vocmc = regex.sub(f":{regex.escape(k)}=.*", f":{k}={v}", vocmc)
        
    # Write various voc versions to separate .voc files
    for fname, suffix in zip([vocmain, vocod, vocinit, vocfull, vocsens, vocmc], 
                             ['m', 'o', 'i', 'f', 's', 'mc']):
        with open(f"{vocname[:-4]}_{suffix}.voc", 'w') as f:
            f.writelines(fname)


def split_lk_vocs(vocname, lkdict):
    """Splits .VOC file into multiple versions for loop knockout tests 
    based on `lkdict`; one version for each dict key, setting all vars 
    in list matching key to 1e-06; also create matching .cin file"""
    for key, varlist in lkdict.items():
        with open(vocname,'r') as f0:
            voclk = f0.read()

        for var in varlist:
            # Select and replace entire line containing var name for each var
            lineregex = r'.*(?<=[^\w ]|\n)\s?' + regex.escape(var) + r'\s?(?=[^\w ]).*'
            voclk = regex.sub(lineregex, f"1e-06<={var}<=1e-06", voclk) # Fix var at 1e-06

        with open(f"{vocname[:-4]}_lk_{key}.voc", 'w') as f1:
            f1.writelines(voclk)
        
        # Also create .cin file for use in normal runs
        cinlines = [f'{var} = 1e-06\n' for var in varlist]
        with open(f"LK_{key}.cin", 'w') as f2:
            f2.writelines(cinlines)
            

def split_vpd(vpdname):
    """Splits .vpd file into main and overdose PMC versions"""
    with open(vpdname,'r') as f0:
        filedata = f0.read()
    
    vpdod = filedata.replace('Elm', 'ODElm')
        
    # Write various vpd versions to separate .vpd files
    for fname, suffix in zip([vpdod], ['o']):
        with open(f"{vpdname[:-4]}_{suffix}.vpd", 'w') as f:
            f.write(fname)


def check_zeroes(scriptname):
    """Check if an .out file has any parameters set to zero (indicates 
    Vengine error), return True if any parameters zeroed OR if # runs = 
    # restarts, and False otherwise"""
    filename = f"{scriptname}.out"
    with open(filename,'r') as f0:
        filedata = f0.readlines()
    
    checklist = []
    for line in filedata:
        if line[0] != ':': # Include only parameter lines
            if ' = 0 ' in line:
                checklist.append(True)
            else:
                checklist.append(False)
        elif ':RESTART_MAX' in line:
            restarts = regex.findall(r'\d+', line)[0]
    
    # Ensure number of simulations != number of restarts
    if f"After {restarts} simulations" in filedata[0]:
        checklist.append(True)
    
    return any(checklist)


def read_payoff(outfile, line=1):
    """Identifies payoff value from .OUT or .REP file - 
    use line 1 (default) for .OUT, or use line 0 for .REP"""
    with open(outfile, 'r') as f:
        payoffline = f.readlines()[line]
    payoffvalue = [float(s) for s in 
                   regex.findall(r'-?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?', payoffline)][0]
    return payoffvalue


def compare_payoff(scriptname):
    """Returns the difference in payoffs between .OUT and .REP file, 
    which should be zero in most cases except when MCMC bugs out"""
    difference = read_payoff(f"{scriptname}.out") - read_payoff(f"{scriptname}.rep", 0)
    print(".OUT and .REP payoff difference is", difference)
    return difference


def increment_seed(vocfile, logfile):
    """Increments random number seed in a .VOC file by 1"""
    with open(vocfile, 'r') as f:
        vocdata = f.read()
    seedregex = regex.compile(r':SEED=\d+')
    try:
        i = int(regex.search(r'\d+', regex.search(seedregex, vocdata).group()).group())
        newdata = seedregex.sub(f":SEED={i+1}", vocdata)
        with open(vocfile, 'w') as f:
            f.write(newdata)
    except:
        write_log("No seed found, skipping incrementing.", logfile)


def get_value(file, varname):
    """General purpose function for reading values from .mdl, .out, etc. 
    files; returns value matching `varname` in a 'var = val' syntax"""
    varregex = regex.compile(r'(?<=([^\w ]|\n)\s?' + regex.escape(varname)
                             + r'\s*=)\s*-?(?:\d*)(\.\d*)?([eE][+\-]?\d+)?')

    with open(file, 'r') as f:
        filetext = f.read()
        value = float((regex.search(varregex, filetext))[0])

    return value


def read_outvals(filename):
    """Read .out file and return matching lists of varnames and values; 
    NOTE need to include file extension in `filename`; also only works 
    for .out files with lower bounds specified"""
    with open(filename, 'r') as f:
        output = [line for line in f.readlines() if line[0] != ':']
        
        varnames = [line.split(' <= ')[1].split(' = ')[0].strip() for line in output]
        basevals = [float(line.split(' <= ')[1].split(' = ')[1]) for line in output]
        
    return varnames, basevals


def compile_sensoutvals(baserunname, sensdict, name, sfxs=['']):
    """Compile dataframe / tabfile of estimated parameter values read 
    from .out files for each of multiple sensitivity runs as specified 
    in `sensdict`, reading from runnames specified with `name` and any 
    `sfxs` (remember to include underscores if needed)"""
    # Read base param values and set up dataframe
    varnames, basevals = read_outvals(f"{baserunname}_main_full.out")
    paramdf = pd.DataFrame({'Value': basevals}, index=varnames)

    # Read iterator from sensdict
    try:
        keys = sensdict.keys()
    except AttributeError:
        keys = sensdict
    
    # Loop through scenarios / runs, with sfxs if needed
    for key in keys:
        for sfx in sfxs:
            varnames, basevals = read_outvals(f"{baserunname}_{name}_{key}{sfx}.out")
            sensdf = pd.DataFrame({f'{key}{sfx}': basevals}, index=varnames)
            
            # Add values from current scenario / run to dataframe
            paramdf = paramdf.merge(sensdf, how='left', left_index=True, right_index=True)

    paramdf.to_csv(f"./{baserunname}_{name}_params.tab", sep='\t')
        
    
def export_fits(baserunname, fitlist, base='main_full', name='main'):
    """Write .cmd file to cleanly export only fit-to-data variables 
    (SimVar, DataVar, NormErr) for elements specified in `fitlist`"""
    savelisttext = [f'SimVar[{var[0]}]\nDataVar[{var[0]}]\nNormErr[{var[0]}]\n'
                    for var in fitlist]

    with open("FitList.lst", 'w') as f:
        f.writelines(savelisttext)
    exporttext = []
    exporttext.append("SPECIAL>NOINTERACTION\n")
    exporttext.append(f"MENU>VDF2TAB|{baserunname}_{base}.vdf|"
                      f"{baserunname}_{name}_fits_raw.tab|FitList.lst||0.25|\n")
    exporttext.append("MENU>EXIT\n")

    with open("ExportFits.cmd", 'w') as f:
        f.writelines(exporttext)


def clean_fits(baserunname, name='main'):
    """Clean fit-to-data tabfile generated using `export_fits` .cmd"""
    cols = pd.read_csv(f"{baserunname}_{name}_fits_raw.tab", sep='\t', index_col=0, 
                       nrows=1, error_bad_lines=False).columns
    res = pd.read_csv(f"{baserunname}_{name}_fits_raw.tab", sep='\t', names=cols, 
                      index_col=0, error_bad_lines=False)
    res.drop('Time', inplace=True)
    res.to_csv(f"./{baserunname}_{name}_fits.tab", sep='\t')
        
        
def downsample(scriptname, samplefrac, remove=True):
    """Downsamples an MCMC _sample tab file according to `samplefrac`, 
    then deletes MCMC _sample and _points files to free up disk space"""
    rawdf = pd.read_csv(f"{scriptname}_MCMC_sample.tab", sep='\t')
    newdf = rawdf.sample(frac=samplefrac)
    newdf.to_csv(f"{scriptname}_MCMC_sample_frac.tab", sep='\t', index=False)
    if remove:
        os.remove(f"{scriptname}_MCMC_sample.tab")
        os.remove(f"{scriptname}_MCMC_points.tab")

    
def merge_samples(baserunname, namelist, name='main', sfx=''):
    """Combines multiple downsampled MCMC outputs into a single 
    sensitivity input tabfile, and writes corresponding .vsc file"""
    filelist = [f"{baserunname}_{n}_MC{sfx}_MCMC_sample_frac.tab" for n in namelist]
    dflist = []
    
    for f in filelist:
        filedf = pd.read_csv(f, sep='\t')
        dflist.append(filedf)
    
    sensdf = pd.concat(dflist, axis=1)
    sensdf.dropna(axis=1, how='all', inplace=True)
    sensdf.dropna().to_csv(f"{baserunname}_{name}_sample_frac{sfx}.tab", sep='\t', index=False)

    # Generate file input method .vsc file, reading from sample
    with open(f"{baserunname}_full{sfx}.vsc", 'w') as f:
        f.write(f",F,,{baserunname}_{name}_sample_frac{sfx}.tab,0")


def recalc_weights(scriptname, fitlist, weights, wtsname='CalWtsAdj.cin', dto=False):
    """Recalculate calibration weights based on residuals from previous 
    calibration round, for all elms in `fitlist`; read existing weights 
    from `weights` and output to `wtsname`; if `dto` is specified, de-
    trend residuals before resampling (use with caution)"""
    t = pd.read_csv(f'{scriptname}.tab', sep='\t', index_col=0)
    t = t[t.columns[::16]]

    # Calculate residuals for each variable
    for var in fitlist:
        t.loc[f'StDev[{var[0]}]'] = t.loc[f'SimVar[{var[0]}]'] - t.loc[f'DataVar[{var[0]}]']
        t.replace({0: np.nan}, inplace=True)

    if dto != False: # Detrend residuals using fitted polynomial of order `dto` (usually 1 or 2)
        for var in fitlist:
            t.loc[f'StDev[{var[0]}]'] = detrend(t.loc[f'StDev[{var[0]}]'].dropna(), order=dto)
        
    t = t.loc[[f'StDev[{var[0]}]' for var in fitlist]]
    stdevs = t.std(axis=1) # Take standard deviations of residuals

    with open(weights, 'r') as f0: # Read in any additional weight modifiers from original file
        tail = [line for line in f0.readlines() 
                if all(f'StDev[{var[0]}]' not in line for var in fitlist)]

    newwts = [f'{k}={v}\n' for k, v in stdevs.items()] + tail

    with open(wtsname, 'w') as f1:
        f1.writelines(newwts)
        
        
def generate_noise(baserunname, fitlist, yearsubs=range(2,33), name='main', sfx='', dto=False):
    """Calculate and resample normalised residuals for vars in `fitlist` 
    based on a fitted multivariate normal distribution, over yearsubs as 
    specified numerically in `yearsubs`, and generate sensitivity input 
    tabfile containing year x var normalised noise values with matching 
    .vsc file; note the tabfile may be large and generating may take a 
    while; if `dto` is specified, detrend normalised residuals before 
    resampling to remove systematic bias / error (use with caution)"""
    table = pd.read_csv(f'{baserunname}_{name}_fits{sfx}.tab', sep='\t', index_col=0)
    table = table[table.columns[::4]]
    
    if dto == False:
        table = table.loc[[f'NormErr[{var[0]}]' for var in fitlist]]

    # Detrend residuals using fitted polynomial of order `dto` (usually 1 or 2)
    else:
        for var in fitlist:
            table.loc[f'NormErrDT[{var[0]}]'] = detrend(
                table.loc[f'NormErr[{var[0]}]'].dropna(), order=dto)
        table = table.loc[[f'NormErrDT[{var[0]}]' for var in fitlist]]
    
    sample = pd.read_csv(f'{baserunname}_{name}_sample_frac{sfx}.tab', sep='\t')

    # Calculate means and covariance matrix for normalised residuals
    means = table.mean(axis=1)
    covs = table.T.cov().astype('float64')

    # Generate flattened column list of yearsubs x fitlist vars
    cols = [[f'RepErrRaw[{var[0]},Y{year:02d}]' for var in fitlist] for year in yearsubs]
    cols_flat = [i for sub in cols for i in sub]

    # Draw and flatten new residual sets from multivariate normal distribution
    rng = np.random.default_rng()
    noise_raw = rng.multivariate_normal(means, covs, (len(sample.index), len(yearsubs)))
    noise_flat = [[i for sub in draw for i in sub] for draw in noise_raw]
    noise = pd.DataFrame(noise_flat, columns=cols_flat)
    noise.clip(-0.99, inplace=True) # Truncate noise values less than -1

    # Merge new residuals with sensitivity input tabfile and save
    merged = pd.concat([sample, noise], axis=1)

    merged.to_csv(f'{baserunname}_{name}_sample_frac_noise{sfx}.tab', sep='\t', index=False)

    # Write corresponding new .vsc file
    with open(f"{baserunname}_full{sfx}.vsc", 'w') as f:
        f.write(f",F,,{baserunname}_{name}_sample_frac_noise{sfx}.tab,0")
        

def generate_param_intervals(baserunname, perc_list, base='main_full', name='main', sfx=''):
    """Compile dataframe / tabfile of estimated parameter values from 
    .out and quantiles from MCMC subsample specified in `perc_list`"""
    samdf = pd.read_csv(f"{baserunname}_{name}_sample_frac{sfx}.tab", sep='\t')

    quants = samdf.quantile(perc_list).T
    quants.index = quants.index.str.strip() # Necessary to strip any trailing spaces

    varnames, basevals = read_outvals(f"{baserunname}_{base}.out")
    
    paramdf = pd.DataFrame({'Value': basevals}, index=varnames)
    paramdf = paramdf.merge(quants, how='left', left_index=True, right_index=True)
    
    paramdf.to_csv(f"{baserunname}_params{sfx}.tab", sep='\t')

    
def clean_sens_raw(senstab, dest=None, remove=True):
    """Do basic reformat and save output tabfile from sensitivity run"""
    sens = pd.read_csv(senstab, sep='\t')

    senstable = pd.pivot_table(sens, index='Time', columns='Percentile')
    senstable = senstable.T
    senstable.index.names = ['Var', 'Perc']
    senstable = senstable[senstable.columns[::4]] # Switch from 1/16 time step to 1/4

    senstable.to_csv(f'{senstab[:-4]}_clean.tab', sep='\t')
    
    if dest: # If specified, copy clean file to destination directory
        copy(f'./{senstab[:-4]}_clean.tab', dest)
        
    if remove: # Unless turned off, delete sensitivity VDF to free up space
        os.remove(f"{senstab[:-4]}.vdf")
    del sens


def clean_full_sens(senstab, polvars, dest=None, remove=True):
    """Do basic reformat and save output tabfile from sensitivity run 
    with full simulation sample saved, saving only `polvars`"""
    colnames = ['Simulation', 'Time'] + polvars # List only relevant columns to subset
    sens = pd.read_csv(senstab, sep='\t', usecols=colnames)

    senstable = pd.pivot_table(sens, index='Time', columns='Simulation')
    senstable = senstable.T
    senstable.index.names = ['Var', 'Run']
    senstable = senstable[senstable.columns[::4]] # Switch from 1/16 time step to 1/4

    senstable.to_csv(f'{senstab[:-4]}_clean.tab', sep='\t')

    if dest: # If specified, copy clean file to destination directory
        copy(f'./{senstab[:-4]}_clean.tab', dest)

    if remove: # Unless turned off, delete main sensitivity outputs to free up space
        os.remove(senstab)
        os.remove(f"{senstab[:-4]}.vdf")
    del sens        


In [ ]:
# Read specified controlfile and unpack into variables
controlfilename = input("Enter control file name (with extension):")
cf = json.load(open(controlfilename, 'r'))

for k,v in cf.items():
    exec(k + '=v')

for setting in [analysissettings]:
    for k, v in setting.items():
        exec(k + '=v')

In [ ]:
##### MAIN ITERATIVE CALIBRATION PROCESS AND MCMC #####

# Set up files in run directory and initialise logfile
master = Script(cf)
master.changes.extend(basescens + scenariolist + policylist + testlist)
master.copy_model_files(f"{baserunname}_IterCal")
copy(f"../{controlfilename}", "./")
copy(f"../{graphs}", "./")
copy(f"../{polsavelist}", "./")
basedir = os.getcwd()
logfile = f"{basedir}/{baserunname}.log"
write_log(f"-----\nStarting new log at {time.ctime()}\nReady to work!", logfile)

# Initialise necessary .voc and .vpd files
split_voc(simsettings['optparm'], fractolfactor, odlinekey, mcsettings)
split_vpd(simsettings['payoff'])

# Define encapsulation function for core calibration process
def iterative_calibration():
    # First do initial calibration round
    compile_script(cf, SubdirScript, 'od', 0, {'optparm': '_o', 'payoff': '_o'}, 
                   logfile, subdir='OD')
    payoff_list = [compile_script(cf, SubdirScript, 'main', 0, {'optparm': '_m'}, 
                                  logfile, chglist=[('od', 0)], subdir='Main')]
    payoff_delta = abs(payoff_list[0])
    compile_script(cf, SubdirScript, 'init', 0, {'optparm': '_i'}, logfile, 
                   chglist=[(['od', 'main'], 0)], subdir='Initials')
    i = 1

    # Then iterate until convergence or until limit is reached
    while payoff_delta > threshold:
        write_log(f"More work? Okay! Starting iteration {i}", logfile)
        compile_script(cf, SubdirScript, 'od', i, {'optparm': '_o', 'payoff': '_o'}, 
                       logfile, chglist=[(['od','main','init'], i-1)], subdir='OD')
        payoff_list.append(
            compile_script(cf, SubdirScript, 'main', i, {'optparm': '_m'}, logfile, 
                           chglist=[(['main','init'], i-1), ('od', i)], subdir='Main'))
        payoff_delta = abs(payoff_list[-1] - payoff_list[-2])
        compile_script(cf, SubdirScript, 'init', i, {'optparm': '_i'}, logfile, 
                       chglist=[('init', i-1), (['od', 'main'], i)], subdir='Initials')
        i += 1

        # Increment random number seeds for VOC files        
        for sfx in ['_o', '_m', '_i']:
            increment_seed(f"{simsettings['optparm'][:-4]}{sfx}.voc", logfile)
        write_log(f"Payoff list thus far is {payoff_list}", logfile)
        write_log(f"Payoff delta is {payoff_delta}", logfile)

        if i > cf['iterlimit']:
            write_log("Iteration limit reached!", logfile)
            break
    else:
        write_log("Payoff delta is less than threshold. Moving on!", logfile)

    # Run one more full calibration with all parameters
    compile_script(cf, LongScript, 'main', 'full', {'optparm': '_f'}, logfile, 
                   chglist=[(['od', 'main', 'init'], i-1)])

# If iterlimit set to 0 (bypass), go straight to all-params Powell optimization
if iterlimit == 0:
    write_log("Iteration is no basis for a system of estimation. Bypassing!", logfile)
    # Skip all-params if previously already done
    if os.path.exists(f"./{baserunname}_main_full.out"):
        if twostep != 0:
            write_log("Starting with the second time!", logfile)
            cf['simsettings']['changes'].append('CalWtsAdj.cin')
        write_log("Hang on to outdated imperialist dogma! Using previous output...", logfile)
    else:
        compile_script(cf, LongScript, 'main', 'full', {'optparm': '_f'}, logfile)
        
        # If two-step calibration needed, recalculate weights and calibrate again
        if twostep != 0:
            write_log("I shall do it a second time!", logfile)
            recalc_weights(f'{baserunname}_main_full', fitlist, 
                           simsettings['changes'][0], dto=noise_dto)
            cf['simsettings']['changes'].append('CalWtsAdj.cin')
            compile_script(cf, LongScript, 'main', 'full', {'optparm': '_f'}, logfile)
            time.sleep(2)

else: # Otherwise run iterative calibration process as normal
    iterative_calibration() # Run encapsulation function defined above
    
    # If two-step calibration needed, recalculate weights and calibrate again
    if twostep != 0:
        write_log("I shall do it a second time!", logfile)
        recalc_weights(f'{baserunname}_main_full', fitlist, 
                       simsettings['changes'][0], dto=noise_dto)
        cf['simsettings']['changes'].append('CalWtsAdj.cin')
        iterative_calibration() # Run iterative calibration process again with new weights
        time.sleep(2)
        
    # Export fit-to-data results only
    export_fits(baserunname, fitlist)
    subprocess.run(f"{vensim7path} \"./ExportFits.cmd\"", check=True)
    clean_fits(baserunname)

# If MCMC option is on, initialise MCMC
if mccores != 0:
    write_log("We're an anarcho-syndicalist commune!\n"
              f"Initiating MCMC at {time.ctime()}!", logfile)
    compile_script(cf, MCScript, 'main', 'MC', {'optparm': '_mc'}, 
                   logfile, chglist=[('main', 'full')])
    write_log(f"MCMC completed at {time.ctime()}!", logfile)
    if twostep != 0:
        cf['simsettings']['changes'].pop() # Remember to remove new weights CIN file!

    # Compile MCMC subsample & calculate quantiles on parameter estimates
    merge_samples(baserunname, ['main']) # Necessary to drop empty column & fix name
    generate_param_intervals(baserunname, param_percs)
    

    # If noise setting is on, generate resampled noise from residuals for MCMC subsample
    if noise != 0:
        yearsubs = [99] + list(range(0, 33)) # NOTE change this as needed to match years included
        generate_noise(baserunname, fitlist, yearsubs=yearsubs, dto=noise_dto)
        write_log("Ready for something completely different!", logfile)

In [ ]:
##### BASELINE PROJECTIONS AND PROJECTION ASSUMPTIONS SENSITIVITY #####

# Switch to Scenarios subfolder & copy necessary files
os.chdir(basedir)
os.makedirs('./Results', exist_ok=True)
master.copy_model_files("Scenarios")
for file in [f"{controlfilename}", f"{baserunname}_main_full.out", 
             f"{baserunname}_full.vsc", f"{baserunname}_main_sample_frac.tab", 
             f"{baserunname}_main_sample_frac_noise.tab"]:
    copy(f"../{file}", "./")

# Run basic projections & scenarios with basic & sensitivity runs
for cin in (basescens + scenariolist):
    copy(f"../{cin}", "./")
    chglist = [('main', 'full'), cin]
    write_log(f"Running scenario {cin}!", logfile)
    compile_script(cf, ScenRunScript, 'final', cin[:-4], {}, logfile, chglist=chglist)
    compile_script(cf, ScenSensScript, 'sens', cin[:-4], {}, logfile, chglist=chglist)
    time.sleep(2)
    clean_sens_raw(f'{baserunname}_sens_{cin[:-4]}.tab', '../Results')
    copy(f'./{baserunname}_final_{cin[:-4]}.tab', '../Results')

# Run element-wise tests of sensitivity to projection assumptions
for cin in basescens[0:2]: # Limit to just main two base scenarios
    for proj in proj_subs:
        # Create _chg cin files to test single proj element variations
        with open(cin, 'r') as f0:
            lines = f0.readlines()

        # Read existing value of switch and reverse it for proj subscript element
        switchval = [line.split('=')[1] for line in lines 
                     if "Switch for constant projections[Proj]" in line]
        lines.append(f'\nSwitch for constant projections[{proj}]={abs(int(switchval[0])-1)}\n')
        with open(f'{cin[:-4]}_chg.cin', 'w') as f1:
            f1.writelines(lines)
        
        # Run basic and sensitivity runs with assumption test .cin
        scen = f'{cin[:-4]}{proj}'
        chglist = [('main', 'full'), f'{cin[:-4]}_chg.cin']
        write_log(f"Running scenario {scen}!", logfile)
        compile_script(cf, ScenRunScript, 'final', scen, {}, logfile, chglist=chglist)
        compile_script(cf, ScenSensScript, 'sens', scen, {}, logfile, chglist=chglist)
        time.sleep(2)
        clean_sens_raw(f'{baserunname}_sens_{scen}.tab', '../Results')
        copy(f'./{baserunname}_final_{scen}.tab', '../Results')
    
os.chdir("..")

print("Ready and waiting!")

In [ ]:
##### POLICY SCENARIOS & SENSITIVITY #####

# Switch to Policies subfolder & copy necessary files
os.chdir(basedir)
os.makedirs('./Results', exist_ok=True)
master.copy_model_files("Policies")
for file in [f"{controlfilename}", f"{baserunname}_main_full.out", 
             
             f"{baserunname}_full.vsc", f"{baserunname}_main_sample_frac.tab", 
             f"{baserunname}_main_sample_frac_noise.tab", polsavelist] + policylist:
    copy(f"../{file}", "./")

# Switch to policy savelist
senssavelist = cf['simsettings']['senssavelist'] # Record regular savelist to switch back later
cf['simsettings']['senssavelist'] = polsavelist

# Run each baseline scenario in combination with each policy 
for cin in basescens:
    for pol in policylist:
        # Run basic and sensitivity runs for each combination
        scen = f'{cin[:-4]}{pol[:-4]}'
        chglist = [('main', 'full'), cin, pol]
        write_log(f"Running scenario {scen}!", 
                  logfile)
        compile_script(cf, ScenRunScript, 'final', scen, {}, logfile, chglist=chglist)
        compile_script(cf, FullSensScript, 'sens', scen,
                       {}, logfile, chglist=chglist) # NOTE: saves full sensitivity sample
        time.sleep(2)
        clean_full_sens(f'{baserunname}_sens_{scen}.tab', polvars, '../Results')
        copy(f'./{baserunname}_final_{scen}.tab', '../Results')

cf['simsettings']['senssavelist'] = senssavelist # Switch back to regular savelist
os.chdir("..")

print("Ready and waiting!")

In [ ]:
##### LOOP KNOCKOUT ANALYSES #####

# Switch to Sensitivity subfolder & copy necessary files
os.chdir(basedir)
master.copy_model_files("Sensitivity")
copy(f"../{controlfilename}", "./")
copy(f"../{baserunname}_main_full.out", "./")

split_lk_vocs(simsettings['optparm'], lkdict) # Create necessary .vocs for loop knockout tests

# Iterate through sets of loops / vars to inactivate as specified in lkdict
for key in lkdict.keys():
    write_log(f"It's {time.ctime()}! Time for a new... idiom!", logfile)
    
    # First run crude (no re-estimation) loop knockout projection using .cin
    compile_script(cf, ScenRunScript, 'lk_run', f'{key}_{basescens[0][:-4]}', {}, logfile,
                   chglist=[('main', 'full'), basescens[0], f'LK_{key}.cin'])
    time.sleep(2)
    
    # Then re-estimate with loops deactivated, and re-run projections
    compile_script(cf, LongScript, 'lk', key, {'optparm': f'_lk_{key}'}, logfile)
    time.sleep(2)
    
    # If two-step calibration needed, recalculate weights and calibrate again
    if twostep != 0:
        recalc_weights(f'{baserunname}_lk_{key}', fitlist, 
                       simsettings['changes'][0], wtsname=f'CalWtsAdj_{key}.cin')
        cf['simsettings']['changes'].append(f'CalWtsAdj_{key}.cin')
        compile_script(cf, LongScript, 'lk', key, {'optparm': f'_lk_{key}'}, logfile)
        time.sleep(2)
        cf['simsettings']['changes'].pop() # Remember to remove new weights CIN file!

    compile_script(cf, ScenRunScript, 'lk', f'{key}_{basescens[0][:-4]}', 
                   {}, logfile, chglist=[('lk', key), basescens[0]]) # Re-run baseline projections
    time.sleep(2)

compile_sensoutvals(baserunname, lkdict, 'lk') # Compile summary sensitivity table
write_log(f"That's not right for my idiom! Leaving at {time.ctime()}!", logfile)

In [ ]:
##### PARAMETRIC ASSUMPTION ELASTICITY ANALYSES #####

# Switch to Sensitivity subfolder & copy necessary files
os.chdir(basedir)
master.copy_model_files("Sensitivity")
copy(f"../{controlfilename}", "./")
copy(f"../{baserunname}_main_full.out", "./")
copy(f"../{simsettings['optparm'][:-4]}_f.voc", "./")

# Compile dict of short form names from first letters of sensvars
sensdict = dict([[''.join([w[0] for w in regex.findall(r"[\w']+", var)]), var] 
                 for var in sensvars])

for key, var in sensdict.items():
    baseval = get_value(simsettings['model'], var) # Get base value directly from .mdl
    
    # Create low and high value assumption .cin files
    with open(f'{key}_L.cin', 'w') as f:
        f.write(f'{var}={baseval*(1 - sensrange)}')
    with open(f'{key}_H.cin', 'w') as f:
        f.write(f'{var}={baseval*(1 + sensrange)}')
    
    write_log(f"It's {time.ctime()} - how do we know if {var} is made of wood?", logfile)
    
    # Run each scenario with re-estimation first then projections
    for scen in [f'{key}_L', f'{key}_H']:
        cf['simsettings']['changes'].append(f'{scen}.cin')
        compile_script(cf, LongScript, 'assm', scen, {'optparm': '_f'}, 
                       logfile, chglist=[('main', 'full')])
        time.sleep(2)
        
        # If two-step calibration needed, recalculate weights and calibrate again
        if twostep != 0:
            recalc_weights(f'{baserunname}_assm_{scen}', fitlist, 
                           simsettings['changes'][0], wtsname=f'CalWtsAdj_{scen}.cin')
            cf['simsettings']['changes'].append(f'CalWtsAdj_{scen}.cin')
            compile_script(cf, LongScript, 'assm', scen, {'optparm': '_f'}, 
                           logfile, chglist=[('main', 'full')])
            time.sleep(2)
            cf['simsettings']['changes'].pop() # Remember to remove new weights CIN file!
        
        compile_script(cf, ScenRunScript, 'assm', f'{scen}_{basescens[0][:-4]}', {}, 
                       logfile, chglist=[('assm', scen), basescens[0]]) # Run baseline projection
        time.sleep(2)
        cf['simsettings']['changes'].pop() # Remember to remove assumption CIN file!

# Compile summary sensitivity table
compile_sensoutvals(baserunname, sensdict, 'assm', sfxs=['_L', '_H'])
write_log(f"It's {time.ctime()}! Burn them!", logfile)

In [ ]:
##### ALTERNATIVE DATA CONDITION ANALYSES #####

# Switch to Sensitivity subfolder & copy necessary files
os.chdir(basedir)
master.copy_model_files("Sensitivity")
copy(f"../{controlfilename}", "./")
copy(f"../{baserunname}_main_full.out", "./")
copy(f"../{simsettings['optparm'][:-4]}_f.voc", "./")
for file in testlist:
    copy(f"../{file}", "./")

# Run calibration for each alternative data condition scenario
for cin in testlist:
    cf['simsettings']['changes'].append(cin)
    compile_script(cf, LongScript, 'test', cin[:-4], {'optparm': '_f'}, 
                   logfile, chglist=[('main', 'full')])
    time.sleep(2)
    
    # If two-step calibration needed, recalculate weights and calibrate again
    if twostep != 0:
        recalc_weights(f'{baserunname}_test_{cin[:-4]}', fitlist, 
                       simsettings['changes'][0], wtsname=f'CalWtsAdj_{cin[:-4]}.cin')
        cf['simsettings']['changes'].append(f'CalWtsAdj_{cin[:-4]}.cin')
        compile_script(cf, LongScript, 'test', cin[:-4], {'optparm': '_f'}, 
                       logfile, chglist=[('main', 'full')])
        time.sleep(2)
        cf['simsettings']['changes'].pop() # Remember to remove new weights CIN file!    
    
    compile_script(cf, ScenRunScript, 'test', f'{cin[:-4]}_{basescens[0][:-4]}', {}, 
                   logfile, chglist=[('test', cin[:-4]), basescens[0]]) # Run baseline projection

    time.sleep(2)
    cf['simsettings']['changes'].pop() # Remember to remove assumption CIN file!

# Compile summary sensitivity table
compile_sensoutvals(baserunname, [cin[:-4] for cin in testlist], 'test')

In [ ]:
##### HOLDOUT DATA ANALYSIS #####

# Switch to Sensitivity subfolder & copy necessary files
os.chdir(basedir)
master.copy_model_files("Sensitivity")
copy(f"../{controlfilename}", "./")
copy(f"../{simsettings['optparm'][:-4]}_f.voc", "./")
copy(f"../{simsettings['optparm'][:-4]}_mc.voc", "./")

# Run initial calibration using data up to `holdoutyear`
write_log(f"At {time.ctime()}, none shall pass after {holdoutyear}!", logfile)
compile_script(cf, LongScript, 'hold', 'full', {'optparm': '_f'}, 
               logfile, setvals=[('MaxDataTime', holdoutyear)])
time.sleep(2)
    
# If two-step calibration needed, recalculate weights and calibrate again
if twostep != 0:
    recalc_weights(f'{baserunname}_hold_full', fitlist, 
                   simsettings['changes'][0], wtsname='CalWtsAdj_hold.cin')
    cf['simsettings']['changes'].append('CalWtsAdj_hold.cin')
    compile_script(cf, LongScript, 'hold', 'full', {'optparm': '_f'}, 
                   logfile, setvals=[('MaxDataTime', holdoutyear)])
    time.sleep(2)

# Export fit-to-data results only
export_fits(baserunname, fitlist, base='hold_full', name='hold')
subprocess.run(f"{vensim7path} \"./ExportFits.cmd\"", check=True)
clean_fits(baserunname, name='hold')

# Run MCMC following initial calibration
compile_script(cf, MCScript, 'hold', 'MC', {'optparm': '_mc'}, 
               logfile, chglist=[('hold', 'full')], setvals=[('MaxDataTime', holdoutyear)])
write_log(f"Just a flesh wound. MCMC completed at {time.ctime()}!", logfile)
if twostep != 0:
    cf['simsettings']['changes'].pop() # Remember to remove new weights CIN file!

# Compile MCMC subsample & calculate quantiles on parameter estimates
merge_samples(baserunname, ['hold'], name='hold') # Necessary to drop empty column & fix name

# Generate resampled noise from residuals for MCMC subsample
yearsubs = [99] + list(range(0, 33)) # NOTE change this as needed to match years included
generate_noise(baserunname, fitlist, yearsubs=yearsubs, name='hold', dto=1)

# Run baseline projection and sensitivity
compile_script(cf, ScenRunScript, 'hold', f'final_{basescens[0][:-4]}', {}, 
               logfile, chglist=[('hold', 'full'), basescens[0]])
compile_script(cf, ScenSensScript, 'hold', f'sens_{basescens[0][:-4]}', {}, logfile, 
               chglist=[('hold', 'full'), basescens[0]], setvals=[('NoiseStartTime', holdoutyear), ('Switch for historical noise', 1)])
time.sleep(2)

clean_sens_raw(f'{baserunname}_hold_sens_{basescens[0][:-4]}.tab', '../Results', remove=False)
copy(f'./{baserunname}_hold_final_{basescens[0][:-4]}.tab', '../Results')

In [ ]:
##### SYNTHETIC DATA GENERATION AND ANALYSES #####

os.chdir(basedir)

# Create syndata generation sample from main noise dataframe
rawdf = pd.read_csv(f"{baserunname}_main_sample_frac_noise.tab", sep='\t')
syndatdf = rawdf.sample(synsample) # Randomly subsample `synsample` param & noise sets
del rawdf
syndatdf.index = range(synsample)
syndatdf.columns = syndatdf.columns.str.strip() # Necessary to strip any trailing spaces
syndatdf.to_csv(f"{baserunname}_syndata_sample.tab", sep='\t', index=False)

# Switch to SynData subfolder & copy necessary files
master.copy_model_files("SynData")

for file in [f"{controlfilename}", f"{simsettings['optparm'][:-4]}_f.voc", 
             f"{simsettings['optparm'][:-4]}_mc.voc"]:
    copy(f"../{file}", "./")

# Create model version to read syndata as data
synmodel = simsettings['model'][:-4] + '_Syn.mdl'
create_syn_mdl(simsettings['model'], synmodel)
syncf = json.load(open(controlfilename, 'r'))

syn_res_list = [] # Initialise container for results of each syndata run

for i in range(synsample):
    # Create .cin files for syndata generation from sample
    values = [f'NoiseStartTime = {styear}\n', 'Switch for historical noise = 1\n']
    values.extend([f'{k} = {v}\n' for k,v in syndatdf.loc[i].to_dict().items()])
    with open(f'SynData{i}.cin', 'w') as f:
        f.writelines(values)
    
    # Run base model with 'true' syndata values from .cin to generate syndata
    write_log(f"Building a large wooden rabbit, round {i}!", logfile)
    compile_script(cf, ScenRunScript, 'syndata', i, {}, logfile, chglist=[f'SynData{i}.cin'])
    time.sleep(2)
    
    # Run estimation and MCMC on synthetic data
    write_log("Now we leap out of the rabbit...", logfile)
    syncf['simsettings']['data'] = [f'{baserunname}_syndata_{i}.vdf', 'yearsubs.vdf']
    compile_script(syncf, LongScript, 'syn', i, {'model': '_Syn', 'optparm': '_f'}, logfile)
    time.sleep(2)

    # If two-step calibration needed, recalculate weights and calibrate again
    if twostep != 0:
        recalc_weights(f'{baserunname}_syn_{i}', fitlist, 
                       simsettings['changes'][0], wtsname=f'CalWtsAdj_{i}.cin')
        cf['simsettings']['changes'].append(f'CalWtsAdj_{i}.cin')
        compile_script(syncf, LongScript, 'syn', i, {'model': '_Syn', 'optparm': '_f'}, logfile)
        time.sleep(2)
    
    write_log(f"...and at {time.ctime()}, take them by surprise!", logfile)
    compile_script(syncf, MCScript, 'syn', f'MC_{i}', {'model': '_Syn', 'optparm': '_mc'}, 
                   logfile, chglist=[('syn', i)])
    write_log(f"Surprise completed at {time.ctime()}!", logfile)
    if twostep != 0:
        cf['simsettings']['changes'].pop() # Remember to remove new weights CIN file!
    
    # Generate CrIs for estimated values
    merge_samples(baserunname, ['syn'], sfx=f'_{i}') # Necessary to drop empty column & fix name
    generate_param_intervals(baserunname, syn_percs, base=f'syn_{i}', name='main', sfx=f'_{i}')
    
    # Combine estimated params and CrIs with 'true' syndata values from original sample
    paramdf = pd.read_csv(f'{baserunname}_params_{i}.tab', sep='\t', index_col=0)
    resdf = pd.concat([syndatdf.loc[i], paramdf], axis=1)
    resdf.dropna(inplace=True)
    resdf.rename(columns={i: 'True'}, inplace=True) # Label 'true' param values
    resdf = resdf.T
    
    syn_res_list.append(resdf)

# Compile and export combined results of all syndata estimations w/ CIs
synresdf = pd.concat(syn_res_list, keys=range(synsample), names=['Run', 'Perc'])
synresdf.to_csv(f'{baserunname}_syndata_results.tab', sep='\t')
copy(f'./{baserunname}_syndata_results.tab', '../Results')
display(synresdf)

In [ ]:
##### CALCULATE PSRF STATISTICS #####

os.chdir(basedir)

name = 'main' # Change if main run name is different

# Compile .cmd script to convert stats .dat to tabfile
cmdtext = [
    "SPECIAL>NOINTERACTION\n",
    f"MENU>DAT2VDF|{baserunname}_{name}_MC_MCMC_stats.dat\n",
    f"SIMULATE>RUNNAME|{baserunname}_{name}_MC\n",
    f"MENU>VDF2TAB|{baserunname}_{name}_MC_MCMC_stats|{baserunname}_{name}_MC_MCMC_stats|\n",
    "SPECIAL>CLEARRUNS\n",
    "MENU>EXIT\n"
]
    
with open(f"{baserunname}_{name}_PSRF.cmd", 'w') as scriptfile:
    scriptfile.writelines(cmdtext)

# Run .dat conversion with Vensim
while True:
    subprocess.run(f"{vensim7path} \"./{baserunname}_{name}_PSRF.cmd\"")
    time.sleep(1)
    try:
        copy(f"./{baserunname}_{name}_MC_MCMC_stats.tab", "./Results")
        break
    except FileNotFoundError:
        print(f"Help! {name} is being repressed!")
        continue

os.chdir("..")

In [ ]:
000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000